## 🧪 04 - Evaluación Final y Exportación

Este notebook tiene como objetivo evaluar el desempeño final del modelo seleccionado sobre el conjunto de prueba y exportar resultados clave para su posterior uso o despliegue.


### 1. 📦 Carga de datos y modelo

Cargamos el conjunto de prueba (`X_test_scaled`, `y_test`) y el modelo final previamente entrenado y guardado.

In [1]:
import joblib

# Cargar datasets de prueba
X_test_scaled = joblib.load('../data/X_test_scaled.pkl')
y_test = joblib.load('../data/y_test.pkl')

# Cargar el modelo entrenado (ajusta el nombre si es diferente)
model = joblib.load('../models/final_model.pkl')  # ← cambia si usaste otro nombre o ubicación

print("Datos y modelo cargados exitosamente.")

FileNotFoundError: [Errno 2] No such file or directory: '../models/final_model.pkl'

### 2. 🧪 Evaluación del modelo

Evaluamos el rendimiento del modelo con las métricas más relevantes:

- Accuracy
- Precision, Recall, F1-score
- Matriz de confusión
- Curva ROC y AUC

### 3. 📊 Visualización de resultados

Mostramos gráficas de evaluación como:

- Matriz de confusión
- Curva ROC
- Importancia de características (si aplica)

### 4. 💾 Exportación

Exportamos:

- El modelo final (`.pkl` o `.joblib`)
- Las métricas a un archivo `.json` o `.csv`
- Las gráficas como imágenes (`.png` o `.svg`)

### 5. ✅ Conclusiones finales

Incluimos un breve resumen del desempeño del modelo y los próximos pasos sugeridos.